In [1]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.orm.collections import attribute_mapped_collection
from sqlalchemy.ext.associationproxy import association_proxy

Base = declarative_base()

In [2]:
! rm ./test_db3.db

In [3]:
class Borehole(Base):
    """The Boreholes Info table"""
    __tablename__ = 'Boreholes'
    id = Column(String(32), primary_key=True)
    intervals = relationship(
        'Interval',
        collection_class=attribute_mapped_collection('id'),
        cascade='all, delete-orphan')
    intervals_values = association_proxy(
        'intervals', 'value',
        creator=lambda k, v: Interval(id=k, description=v['description'], interval_number=v['interval_number']))

class Interval(Base):
    """The Interval table"""
    __tablename__ = 'Intervals'
    id = Column(Integer, primary_key=True)
    borehole = Column(String(32), ForeignKey('Boreholes.id'))
    # Note that this could be a numeric ID as well
    interval_number = Column(Integer)
    components = relationship('Component',secondary='Linkintervalcomponent')
    description = Column(String(32))
    
class Component(Base):
    """The Component table"""
    __tablename__ = 'Components'
    id = Column(String(32), primary_key=True)
    intervals = relationship(Interval,secondary='Linkintervalcomponent')
    description = Column(String(32))

class LinkIntervalComponent(Base):
    __tablename__ = 'Linkintervalcomponent'

    int_id = Column(
        Integer, 
        ForeignKey('Intervals.id'), 
        primary_key = True)

    comp_id = Column(
       Integer, 
       ForeignKey('Components.id'), 
       primary_key = True)

In [4]:
engine = create_engine('sqlite:///test_db3.db', echo=True)

In [5]:
Base.metadata.create_all(engine)

2020-12-04 15:50:53,998 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-04 15:50:53,999 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:50:54,001 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-04 15:50:54,001 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:50:54,003 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-04 15:50:54,003 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:50:54,005 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-04 15:50:54,006 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:50:54,006 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-04 15:50:54,007 INFO sqlalchemy.engine.base.Engine ()
2020-12-04 15:50:54,008 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-04 15:50:54,008 INFO sqlalchemy.engine.base.Engine ()
2020-12-04

In [6]:
from contextlib import contextmanager

Session = sessionmaker(bind=engine)

@contextmanager
def session_scope():
    """Provide a transactional scope around a series of operations."""
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()

In [7]:
from striplog import Lexicon, Striplog

In [8]:
lexicon = Lexicon.default()

In [9]:
borehole_dict = {'F01':'test.las', 'F02':'test.las'} # boreholes to insert into the db

In [28]:
int_id = 0
bh_id = 0
boreholes = []
comp_id = 0
component_dict={}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number}})
        interval_number+=1
        int_id += 1
    # print(d)
    boreholes[bh_id].intervals_values = d
    bh_id += 1         

In [29]:
component_dict

{Component({'lithology': 'siltstone', 'colour': 'grey'}): 0,
 Component({'lithology': 'anhydrite'}): 1,
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}): 2,
 Component({'lithology': 'dolomite'}): 3,
 Component({'lithology': 'siltstone', 'colour': 'red'}): 4,
 Component({'lithology': 'limestone'}): 5}

In [27]:
strip.components

[Component({'lithology': 'siltstone', 'colour': 'grey'}),
 Component({'lithology': 'anhydrite'}),
 Component({'lithology': 'sandstone', 'grainsize': 'vf-f', 'colour': 'grey'}),
 Component({'lithology': 'dolomite'}),
 Component({'lithology': 'siltstone', 'colour': 'red'}),
 Component({'lithology': 'limestone'})]

In [15]:
strip[0].components

[Component({'lithology': 'anhydrite'})]

In [11]:
boreholes

[<__main__.Borehole at 0x7f84845a0710>, <__main__.Borehole at 0x7f8484540ef0>]

In [12]:
with session_scope() as session:
    for bh in boreholes:
        session.add(bh)

2020-12-04 15:50:54,873 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-04 15:50:54,876 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-04 15:50:54,876 INFO sqlalchemy.engine.base.Engine (('F01',), ('F02',))
2020-12-04 15:50:54,880 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description) VALUES (?, ?, ?, ?)
2020-12-04 15:50:54,881 INFO sqlalchemy.engine.base.Engine ((0, 'F01', 0, 'Anhydrite'), (1, 'F01', 1, 'Sandstone, grey, vf-f'), (2, 'F01', 2, 'Anhydrite'), (3, 'F01', 3, 'Dolomite'), (4, 'F01', 4, 'Anhydrite'), (5, 'F01', 5, 'Sandstone, grey, vf-f'), (6, 'F01', 6, 'Siltstone, red'), (7, 'F01', 7, 'Dolomite')  ... displaying 10 of 50 total bound parameter sets ...  (48, 'F02', 23, 'Limestone'), (49, 'F02', 24, 'Volcanic'))
2020-12-04 15:50:54,883 INFO sqlalchemy.engine.base.Engine COMMIT
